In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('./datasets/DfCleaned.csv')

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import LSTM, Dense
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier

2023-11-10 13:51:08.405180: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 13:51:08.577791: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 13:51:08.577826: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 13:51:08.579009: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-10 13:51:08.668268: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 13:51:08.669116: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

### Model n-gram3

In [3]:
max_features = 1000
tokenizer = Tokenizer(num_words= max_features, split= ' ')
tokenizer.fit_on_texts(df['ngram3'].values)
x = tokenizer.texts_to_sequences(df['ngram3'].values)
x = pad_sequences(x)

In [4]:
y = pd.get_dummies(df['clarity']).values

In [5]:
from sklearn.model_selection import KFold

In [6]:
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience= 3)

In [7]:
def modelLSTM():
    max_features = 1000
    embed_dim = 32
    lstm_out = 32
    
    model = Sequential([
        layers.Embedding(max_features, embed_dim, input_length= x.shape[1]),
        layers.Conv1D(filters=32, kernel_size=3, padding='same', activation= 'relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.LSTM(128, dropout=0.1, recurrent_dropout=0.5),
        layers.Dense(64, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [10]:
clf = KerasClassifier(build_fn=modelLSTM, epochs=10, batch_size=32, verbose=0)

In [11]:
result = cross_val_score(clf, x, y, scoring='accuracy', cv=3)
print(result)
result.mean()

/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


### Model PoS

In [ ]:
df_pos = pd.read_csv('./datasets/DfPos.csv')

In [ ]:
max_features = 1000
tokenizer = Tokenizer(num_words= max_features, split= ' ')
tokenizer.fit_on_texts(df_pos['pos'].values)
x = tokenizer.texts_to_sequences(df_pos['pos'].values)
x = pad_sequences(x)

In [ ]:
y = pd.get_dummies(df['clarity']).values

In [ ]:
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience= 3)

In [ ]:
def modelLSTM():
    max_features = 1000
    embed_dim = 32
    lstm_out = 32
    
    model = Sequential([
        layers.Embedding(max_features, embed_dim, input_length= x.shape[1]),
        layers.Conv1D(filters=32, kernel_size=3, padding='same', activation= 'relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.LSTM(128, dropout=0.1, recurrent_dropout=0.1),
        layers.Dense(64, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
clf = KerasClassifier(build_fn=modelLSTM, epochs=10, batch_size=32, verbose=1)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
result = cross_val_score(clf, x, y, scoring='accuracy', cv=3)
print(result)
result.mean()

/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/6


2023-11-10 13:10:43.637038: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 43944000 exceeds 10% of free system memory.


94/94 [==============================] - 176s 2s/step - loss: 0.6350 - accuracy: 0.3737
Epoch 2/6
94/94 [==============================] - 171s 2s/step - loss: 0.5883 - accuracy: 0.4797
Epoch 3/6
94/94 [==============================] - 169s 2s/step - loss: 0.5596 - accuracy: 0.5170
Epoch 4/6
94/94 [==============================] - 180s 2s/step - loss: 0.5339 - accuracy: 0.5503
Epoch 5/6
94/94 [==============================] - 179s 2s/step - loss: 0.5091 - accuracy: 0.5800
Epoch 6/6
94/94 [==============================] - 28s 291ms/step


/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/6
94/94 [==============================] - 181s 2s/step - loss: 0.6403 - accuracy: 0.3653
Epoch 2/6
94/94 [==============================] - 179s 2s/step - loss: 0.6196 - accuracy: 0.4110
Epoch 3/6
94/94 [==============================] - 176s 2s/step - loss: 0.5778 - accuracy: 0.5003
Epoch 4/6
94/94 [==============================] - 170s 2s/step - loss: 0.5512 - accuracy: 0.5260
Epoch 5/6
94/94 [==============================] - 167s 2s/step - loss: 0.5322 - accuracy: 0.5547
Epoch 6/6
94/94 [==============================] - 27s 283ms/step


array([0.44033333, 0.44933333])

In [ ]:
#add teste normal